<a href="https://colab.research.google.com/github/kaka20022002/new-2025-11-25/blob/main/MNIST_Digit_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow numpy matplotlib

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# 1. 加载数据
print("正在下载数据...")
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 2. 归一化 (把像素变到 0-1 之间)
x_train, x_test = x_train / 255.0, x_test / 255.0

# 看看数据长什么样（画出前5张图）
plt.figure(figsize=(10,2))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f"Label: {y_train[i]}")
    plt.axis('off')
plt.show()

print("数据准备完毕！")


In [ ]:
# 3. 搭建模型
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), # 拉平
    tf.keras.layers.Dense(128, activation='relu'), # 隐藏层
    tf.keras.layers.Dropout(0.2),                  # 防止死记硬背
    tf.keras.layers.Dense(10, activation='softmax')# 输出层
])

# 4. 编译
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 5. 训练 (训练5轮)
print("开始训练神经网络...")
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [ ]:
# 画出准确率变化曲线
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('How smart is the model?')
plt.legend()
plt.show()

# 评估最终分数
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\n最终准确率: {test_acc*100:.2f}%")

In [ ]:
from google.colab import files
from PIL import Image, ImageOps

# 1. 上传图片
print("请上传一张你手写的数字图片 (白底黑字):")
uploaded = files.upload()

for fn in uploaded.keys():
    # 2. 打开图片并转为灰度图
    image = Image.open(fn).convert('L')

    # 3. 预处理：
    # 因为MNIST是黑底白字，而我们平时写字是白底黑字
    # 所以需要反转颜色 (Invert)，除非你本来就是黑纸白字
    image = ImageOps.invert(image)

    # 调整大小到 28x28 像素
    image = image.resize((28, 28))

    # 转为数组并归一化
    img_array = np.array(image) / 255.0

    # 增加一个维度 (变成 1, 28, 28) 以符合模型输入要求
    img_array = img_array.reshape(1, 28, 28)

    # 4. 预测
    prediction = model.predict(img_array)
    result = np.argmax(prediction)

    # 5. 显示结果
    plt.imshow(img_array[0], cmap='gray')
    plt.title(f"AI Predicts: {result}")
    plt.axis('off')
    plt.show()
    print(f"AI 认为你写的是: {result}")

In [ ]:
from google.colab import files
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt

# 1. 再次上传
print(">>> 请上传下一张图片...")
uploaded = files.upload()

def smart_process_image(image_path):
    """
    智能图像处理函数：
    1. 转灰度
    2. 自动反色（把白纸变黑底）
    3. 二值化（去除阴影）
    4. 自动裁剪（只保留数字部分）
    5. 居中缩放
    """
    # 打开图片转灰度
    img = Image.open(image_path).convert('L')

    # 自动增强对比度
    img = ImageOps.autocontrast(img)

    # 如果是白底黑字（判断左上角像素是否为亮色），则反转为黑底白字
    if img.getpixel((0, 0)) > 128:
        img = ImageOps.invert(img)

    # 二值化：过滤噪点，阈值设为 100
    img = img.point(lambda p: 255 if p > 100 else 0)

    # === 关键步骤：智能裁剪 (Crop to Content) ===
    # 获取数字的边界框（非零像素区域）
    bbox = img.getbbox()
    if bbox:
        img = img.crop(bbox) # 只把数字切出来

    # === 关键步骤：保持比例缩放到盒子中间 ===
    # 创建一个 28x28 的纯黑画布
    new_img = Image.new('L', (28, 28), 0)

    # 把切出来的数字缩放一下，长边限制在 20 像素（留点边距）
    # 这样数字不会撑满整个格子，和 MNIST 数据集保持一致
    img.thumbnail((20, 20), Image.Resampling.LANCZOS)

    # 计算粘贴位置（居中）
    w, h = img.size
    x_offset = (28 - w) // 2
    y_offset = (28 - h) // 2

    # 把数字贴到黑画布中间
    new_img.paste(img, (x_offset, y_offset))

    return new_img

for fn in uploaded.keys():
    # 处理图片
    processed_img = smart_process_image(fn)

    # 转换为模型能读的数组
    img_array = np.array(processed_img) / 255.0
    model_input = img_array.reshape(1, 28, 28)

    # 预测
    prediction = model.predict(model_input)
    result = np.argmax(prediction)
    confidence = np.max(prediction) * 100

    # === 展示结果 ===
    plt.figure(figsize=(6, 3))

    # 左边：显示经过“智能裁剪”后的样子
    plt.subplot(1, 2, 1)
    plt.imshow(img_array, cmap='gray')
    plt.title(f"Processed Input\nAI thinks: {result}")
    plt.axis('off')

    # 右边：显示概率条
    plt.subplot(1, 2, 2)
    plt.bar(range(10), prediction[0], color='skyblue')
    plt.xticks(range(10))
    plt.title(f"Confidence: {confidence:.1f}%")
    plt.ylim(0, 1.1)

    plt.tight_layout()
    plt.show()

    print(f"文件名: {fn}")
    print(f"最终识别结果: {result}")